In [ ]:
from obspy import *
import struct
import os
import numpy as np

def unpackAfile(infile):

# == opening Afile ==
    b= os.path.getsize(infile)
    FH = open(infile, 'rb')
    line = FH.read(b)
    fileHeader= struct.unpack("<4s3h6bh6s", line[0:24])
    
    fileLength = fileHeader[3]
    port = fileHeader[10]
    FirstStn = fileHeader[11][0:4].decode('ASCII').rstrip()
    #print(fileHeader)
# =================================Header===================================
    
    portHeader = []
    for i in range(24,port*32,32):
        port_data = struct.unpack("<4s4s3sbh2b4s12b",line[i:i+32])
        portHeader.append(port_data)

# =================================Data===================================    

    dataStartByte = 24+int(port)*32
    dataPoint = 3*int(port)*int(fileLength)*100
    times = int(port)*3*4
    data=[]

    data = struct.unpack("<%di"%dataPoint,line[dataStartByte:dataStartByte+dataPoint*4])

    
    portHeader = np.array(portHeader)    
    data = np.array(data)    
    idata =data.reshape((3,port,fileLength*100),order='F')
    
#== write to obspy Stream --
    #print(fileHeader)
    #print(len(idata[0][0]))
    sttime = UTCDateTime(fileHeader[1],fileHeader[4],fileHeader[5],fileHeader[6],fileHeader[7],fileHeader[8],fileHeader[2])
    npts = fileHeader[3]*fileHeader[9]
    samp = fileHeader[9]
    #print(sttime)
    # afst = Afile's Stream
    afst = Stream()
    
    for stc in range(fileHeader[10]):
        stn = portHeader[stc][0].decode('ASCII').rstrip()
        instrument = portHeader[stc][1].decode('ASCII').rstrip()
        loc = '0'+str(portHeader[stc][6].decode('ASCII'))
        #net = "TW"
        net = str(portHeader[stc][7].decode('ASCII')).rstrip()
        GPS = int(portHeader[stc][3])
        
        # remove GPS unlock or broken station
        if ( GPS == 1 or GPS == 2 ):
            chc = 0
            if instrument == 'FBA':
                chc = 1
            elif instrument == 'SP':
                chc = 4
            elif instrument == 'BB':
                chc = 7
            
            #print(chc,instrument)
            
            # for each channel in port
            for ch in range(3):
                #print(num,ch,chc)
                chn = 'Ch'+str(chc+ch)
                #print(stc,channel)
                
                stats = {'network': net, 'station': stn, 'location': loc,
                        'channel': chn, 'npts': npts, 'sampling_rate': samp,
                        'starttime': sttime}
                
                data = np.array(idata[ch][stc], dtype=float)
                sttmp = Stream([Trace(data=data, header=stats)])
                afst += sttmp
    
    #stst = stms[0]
    #stst.detrend('linear').plot()
    #stms.write('test.mseed', format='MSEED')    
    
    return afst, FirstStn, fileHeader

In [ ]:
import csv
import codecs
import time

#start_time = time.time()
weather = dict()
weather1 = dict()
with open ('network20.csv', 'w', newline = '',encoding='big5') as csvfile:
    datanames=['network','station','location','channel','npts','sampling_rate','starttime','delta','calib']
    writer = csv.DictWriter(csvfile , fieldnames = datanames)
    #writer.writeheader()
    for j in range(0,5556):
        for i in range(0,36):
            #print(unpackAfile('05290646.40C')[0][i].stats['network'])
            #print(type(unpackAfile('05290646.40C')[0][i].stats['network']))
            net=str(unpackAfile('05290646.40C')[0][i].stats['network'])
            stn=str(unpackAfile('05290646.40C')[0][i].stats['station'])
            loc=str(unpackAfile('05290646.40C')[0][i].stats['location'])
            chn=str(unpackAfile('05290646.40C')[0][i].stats['channel'])
            npts=str(unpackAfile('05290646.40C')[0][i].stats['npts'])
            samp=str(unpackAfile('05290646.40C')[0][i].stats['sampling_rate'])
            sttime=str(unpackAfile('05290646.40C')[0][i].stats['starttime'])
            delt=str(unpackAfile('05290646.40C')[0][i].stats['delta'])
            cal=str(unpackAfile('05290646.40C')[0][i].stats['calib'])
            weather['network']=net
            weather['station']=stn
            weather['location']=loc
            weather['channel']=chn
            weather['npts']=npts
            weather['sampling_rate']=samp
            weather['starttime']=sttime
            weather['delta']=delt
            weather['calib']=cal
            writer.writerow(weather)
            
#創造地震資料

In [ ]:
from pyhive import hive  # or import hive
cursor = hive.connect(host='....',port=10000,auth='LDAP',username='root',password='....', database='default').cursor()
#connect hiveserver2
#host為ubuntu主機名或ip address
#username用建立/user/hive/warehouse的授權名,要不然會出現一些授權上的問題
#database為資料庫
#password是你在執行beeline時所輸入的密碼

In [ ]:
#參考資料hive指令手冊網址https://cwiki.apache.org/confluence/display/Hive/LanguageManual+DDL
cursor.execute('show tables')
print (cursor.fetchall())
#顯示table有哪些

In [ ]:
cursor.execute("CREATE TABLE name (network STRING, station STRING, location STRING, channel STRING, npts STRING, sampling_rate STRING, starttime STRING, delta STRING, calib STRING) row format delimited fields terminated by ',' lines terminated by '\n' stored as textfile")
cursor.execute('show tables')
print (cursor.fetchall())
#創建 table 地震資料的格式

In [ ]:
cursor.execute('drop table name')
cursor.execute('show tables')
print (cursor.fetchall())
#刪除table

In [ ]:
cursor.execute('show databases')
print (cursor.fetchall())
#顯示database有哪些

In [ ]:
cursor.execute('create database test1')
print (cursor.fetchall())
#創建database

In [ ]:
cursor.execute("load data inpath '' into table earthquakedata1")
#load data inpath 是用在hdfs系統中的資料放入在earthquake1 table 中 , 而 ''是放路徑

In [ ]:
cursor.execute("load data local inpath '' into table earthquakedata1")

#load data inpath 是用在ubuntu主機中的資料放入在earthquake1 table 中 , 而 ''是放路徑

In [ ]:
cursor.execute("insert into table earthquakedata2 select * from earthquakedata1")
#插入資料到Table